In [15]:
import pandas as pd
from glob import glob
import os
import sys
cnn_scripts_path = '/home/csutter/DRIVE-clean/CNN/scripts'
sys.path.append(cnn_scripts_path)

# imported reusable code from CNN directory
import results_summaries

### CNN

In [16]:
files_list = glob("/home/csutter/DRIVE-clean/CNN/data_preds/*")
files_list = [x for x in files_list if "A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue" in x]

# OG WAY: A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue
# NEW WAY: A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue
print(len(files_list))

30


In [17]:
# df = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_exp_innerVal.csv")

dfs30 =[]
for f in files_list:
    d = pd.read_csv(f)
    d = d[d["innerPhase"]=="innerVal"] # change this according to phase
    dfs30.append(d)

df = pd.concat(dfs30).reset_index()

print(len(df))
display(df.head(4))



107445


,index,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,innerPhase,Unnamed: 0,outerPhase,img_name,img_orig,site,...,timeofevent,innerPhaseOrig,prob_dry,prob_poor_viz,prob_snow,prob_snow_severe,prob_wet,model_pred,tracker,model_prob
0,459,459,459,459,innerVal,459,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,...,"dawn:11:29:00.847281, sunrise:11:59:10.264774,...",innerVal,0.007299,0.192872,0.012995,0.766698,0.020136,snow_severe,twotrain_OT2_m3_T5V0,0.766698
1,460,460,460,460,innerVal,460,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,...,"dawn:11:29:00.847281, sunrise:11:59:10.264774,...",innerVal,0.010275,0.138785,0.015527,0.817639,0.017774,snow_severe,twotrain_OT2_m3_T5V0,0.817639
2,461,461,461,461,innerVal,461,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,...,"dawn:11:29:00.847281, sunrise:11:59:10.264774,...",innerVal,0.065053,0.009944,0.687876,0.122768,0.114359,snow,twotrain_OT2_m3_T5V0,0.687876
3,462,462,462,462,innerVal,462,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,...,"dawn:11:29:00.847281, sunrise:11:59:10.264774,...",innerVal,0.218485,0.017282,0.389381,0.122555,0.252297,snow,twotrain_OT2_m3_T5V0,0.389381


In [6]:
df.columns

Index(['index', 'Unnamed: 0.1', 'innerPhase', 'outerPhase', 'img_name',
       'img_orig', 'site', 'img_cat', 'foldnum', 'timeofday', 'timeofevent',
       'calib_prob_dry', 'calib_prob_poor_viz', 'calib_prob_snow',
       'calib_prob_snow_severe', 'calib_prob_wet', 'calib_prob', 'Unnamed: 0',
       'img_orig_hrrr', 'img_index', 'Latitude', 'Longitude', 'yr', 'mo',
       'day', 'date', 'time', 'ymd', 'date_and_time_str', 'date_and_time_dt',
       'date_and_time_dt_round', 'date_and_time_dt_init', 'init_hr',
       'init_year', 'init_month', 'init_day', 'init_yyyymmdd', 'hrrr_file',
       'HRRR_latlon', 'HRRR_distkm', 'time.1', 'valid_time', 'latitude',
       'longitude', 't2m', 'pt', 'sh2', 'd2m', 'r2', 'u10', 'v10', 'si10',
       'asnow', 'tp', 'orog', 'cape', 'mslma', 'dswrf', 'dlwrf', 'tcc', 'gh',
       'dpt', 'atmosphere', 'isobaricInhPa', 'uavg', 'ds_prob_dry',
       'ds_prob_poor_viz', 'ds_prob_snow', 'ds_prob_snow_severe',
       'ds_prob_wet', 'ds_pred', 'ds_prob', 'mod

In [18]:
dict_i = results_summaries.calc_summary(dfinput = df, tracker_desc = "", exp_desc = "",  exp_details = f"")

print(dict_i)

df = pd.DataFrame(dict_i,index=[0])


df["overall"] = df["correctims"]/df["totalims"]


for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]


df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

display(df)


{'totalims': 107445, 'correctims': 72552, 'nims_snow_severe': 7635, 'correct_snow_severe': 5953, 'nims_snow': 14945, 'correct_snow': 8992, 'nims_wet': 42290, 'correct_wet': 29556, 'nims_dry': 38000, 'correct_dry': 24251, 'nims_poor_viz': 4575, 'correct_poor_viz': 3800, 'ok': 96587, 'exp_desc': '', 'exp_details': '', 'tracker': ''}


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,...,exp_desc,exp_details,tracker,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
0,107445,72552,7635,5953,14945,8992,42290,29556,38000,24251,...,,,,0.675248,0.638184,0.830601,0.601673,0.779699,0.698889,0.709809


### Downstream

In [19]:
files_list = glob("/home/csutter/DRIVE-clean/downstream/data_preds/*")

files_list = [x for x in files_list if "A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue" in x]

# OG WAY: A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue
# NEW WAY: A_resnet_TRLETrue_ASTFalse_L20_1_DR0_4_E75_AugTrue
print(len(files_list))

print(len(files_list))

30
30


In [20]:
# df = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_exp_innerVal.csv")

dfs30 =[]
for f in files_list:
    d = pd.read_csv(f)
    d = d[d["innerPhase"]=="innerVal"]# change this according to phase
    d["model_pred"] = d["ds_pred"]
    dfs30.append(d)

df = pd.concat(dfs30).reset_index()

print(len(df))
display(df.head(4))


107445


,index,Unnamed: 0.1,innerPhase,outerPhase,img_name,img_orig,site,img_cat,foldnum,timeofday,...,isobaricInhPa,uavg,ds_prob_dry,ds_prob_poor_viz,ds_prob_snow,ds_prob_snow_severe,ds_prob_wet,ds_pred,ds_prob,model_pred
0,921,921,innerVal,inner5CV,I_90_at_Interchange_48_(Batavia)__Eastbound__S...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6493,snow,5,day,...,500.0,4.973033,0.002662,0.007870,0.393987,0.585302,0.010179,snow_severe,0.585302,snow_severe
1,922,922,innerVal,inner5CV,I_90_at_Interchange_48_(Batavia)__Eastbound__S...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6493,snow_severe,5,night,...,500.0,5.784990,0.056839,0.481237,0.068521,0.354823,0.038579,poor_viz,0.481237,poor_viz
2,923,923,innerVal,inner5CV,I_90_at_Interchange_48_(Batavia)__Eastbound__S...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6493,snow_severe,5,night,...,500.0,5.661163,0.000171,0.074690,0.032528,0.892467,0.000144,snow_severe,0.892467,snow_severe
3,924,924,innerVal,inner5CV,I_90_at_Interchange_48_(Batavia)__Eastbound__S...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6493,snow,5,night,...,500.0,5.960207,0.000342,0.048785,0.043448,0.902744,0.004680,snow_severe,0.902744,snow_severe


In [21]:
dict_i = results_summaries.calc_summary(dfinput = df, tracker_desc = "", exp_desc = "",  exp_details = f"")

print(dict_i)

df = pd.DataFrame(dict_i,index=[0])


df["overall"] = df["correctims"]/df["totalims"]


for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]


df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

display(df)


{'totalims': 107445, 'correctims': 86220, 'nims_snow_severe': 7635, 'correct_snow_severe': 6059, 'nims_snow': 14945, 'correct_snow': 11101, 'nims_wet': 42290, 'correct_wet': 31974, 'nims_dry': 38000, 'correct_dry': 33714, 'nims_poor_viz': 4575, 'correct_poor_viz': 3372, 'ok': 101897, 'exp_desc': '', 'exp_details': '', 'tracker': ''}


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,...,exp_desc,exp_details,tracker,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
0,107445,86220,7635,6059,14945,11101,42290,31974,38000,33714,...,,,,0.802457,0.887211,0.737049,0.74279,0.793582,0.756065,0.783339


### Experiments
- Side experiments for manuscript regarding generalizability and other takeaways

#### Experiment 1: Proving validity of downstream flow (training on a diff dataset). This side experiment is then the alternative, which is training on the same dataset for both CNN and RF, calling this oneTrain experiment.

CNN

In [12]:
files_list = glob("/home/csutter/DRIVE-clean/CNN/data_preds_expOneTrain/*")
files_list = [x for x in files_list if "A_resnet_TRLETrue_ASTTrue_L20_1_DR0_2_E75_AugTrue" in x]

print(len(files_list))

30


In [13]:
# df = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_exp_innerVal.csv")

dfs30 =[]
for f in files_list:
    d = pd.read_csv(f)
    d = d[d["innerPhase"]=="innerVal"] # change this according to phase
    dfs30.append(d)

df = pd.concat(dfs30).reset_index()

print(len(df))
display(df.head(4))



107445


,index,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,innerPhase,Unnamed: 0,outerPhase,img_name,img_orig,site,...,timeofevent,innerPhaseOrig,prob_dry,prob_poor_viz,prob_snow,prob_snow_severe,prob_wet,model_pred,tracker,model_prob
0,459,459,459,459,innerVal,459,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,...,"dawn:11:29:00.847281, sunrise:11:59:10.264774,...",innerVal,0.002666,0.050255,0.002159,0.940243,0.004678,snow_severe,nestcv_5cat_onetrain_OT5_m4_T4V0,0.940243
1,460,460,460,460,innerVal,460,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,...,"dawn:11:29:00.847281, sunrise:11:59:10.264774,...",innerVal,0.001996,0.020671,0.002161,0.972410,0.002762,snow_severe,nestcv_5cat_onetrain_OT5_m4_T4V0,0.972410
2,461,461,461,461,innerVal,461,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,...,"dawn:11:29:00.847281, sunrise:11:59:10.264774,...",innerVal,0.037506,0.000784,0.818759,0.103861,0.039091,snow,nestcv_5cat_onetrain_OT5_m4_T4V0,0.818759
3,462,462,462,462,innerVal,462,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,...,"dawn:11:29:00.847281, sunrise:11:59:10.264774,...",innerVal,0.130350,0.002081,0.510149,0.254475,0.102945,snow,nestcv_5cat_onetrain_OT5_m4_T4V0,0.510149


In [14]:
dict_i = results_summaries.calc_summary(dfinput = df, tracker_desc = "", exp_desc = "",  exp_details = f"")

print(dict_i)

df = pd.DataFrame(dict_i,index=[0])


df["overall"] = df["correctims"]/df["totalims"]


for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]


df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

display(df)


{'totalims': 107445, 'correctims': 77104, 'nims_snow_severe': 7635, 'correct_snow_severe': 6239, 'nims_snow': 14945, 'correct_snow': 9608, 'nims_wet': 42290, 'correct_wet': 30787, 'nims_dry': 38000, 'correct_dry': 26821, 'nims_poor_viz': 4575, 'correct_poor_viz': 3649, 'ok': 98746, 'exp_desc': '', 'exp_details': '', 'tracker': ''}


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,...,exp_desc,exp_details,tracker,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
0,107445,77104,7635,6239,14945,9608,42290,30787,38000,26821,...,,,,0.717614,0.705816,0.797596,0.642891,0.817158,0.727997,0.738291


Downstream

In [6]:
files_list = glob("/home/csutter/DRIVE-clean/downstream/data_preds_expOneTrain/*")

print(len(files_list))


30


In [7]:
# df = pd.read_csv("/home/csutter/DRIVE-clean/downstream/data_results/results_by_exp_innerVal.csv")

dfs30 =[]
for f in files_list:
    d = pd.read_csv(f)
    d = d[d["innerPhase"]=="innerVal"]# change this according to phase
    d["model_pred"] = d["ds_pred"]
    dfs30.append(d)

df = pd.concat(dfs30).reset_index()

print(len(df))
display(df.head(4))


107445


,index,Unnamed: 0.1,innerPhase,outerPhase,img_name,img_orig,site,img_cat,foldnum,timeofday,...,isobaricInhPa,uavg,ds_prob_dry,ds_prob_poor_viz,ds_prob_snow,ds_prob_snow_severe,ds_prob_wet,ds_pred,ds_prob,model_pred
0,459,459,innerVal,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,snow_severe,0,night,...,500.0,5.797924,0.000131,0.000979,0.003921,0.994951,0.000017,snow_severe,0.994951,snow_severe
1,460,460,innerVal,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,snow_severe,0,night,...,500.0,5.468119,0.000239,0.000000,0.002644,0.997104,0.000014,snow_severe,0.997104,snow_severe
2,461,461,innerVal,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,snow_severe,0,day,...,500.0,4.567119,0.002370,0.001546,0.982409,0.001933,0.011741,snow,0.982409,snow
3,462,462,innerVal,inner5CV,I_81_at_Exit_50__Northbound__Skyline_6163_2022...,/home/csutter/DRIVE/site_analysis/group_labeli...,Skyline_6163,snow_severe,0,day,...,500.0,4.894132,0.002789,0.000026,0.982128,0.002827,0.012230,snow,0.982128,snow


In [8]:
dict_i = results_summaries.calc_summary(dfinput = df, tracker_desc = "", exp_desc = "",  exp_details = f"")

print(dict_i)

df = pd.DataFrame(dict_i,index=[0])


df["overall"] = df["correctims"]/df["totalims"]


for c in ["dry", "poor_viz", "snow", "snow_severe", "wet"]:
    df[f"perc_{c}"] = df[f"correct_{c}"]/df[f"nims_{c}"]


df["avg_recall"] = df[["perc_dry", "perc_poor_viz", "perc_snow", "perc_snow_severe", "perc_wet"]].mean(axis = 1)

display(df)


{'totalims': 107445, 'correctims': 84301, 'nims_snow_severe': 7635, 'correct_snow_severe': 6059, 'nims_snow': 14945, 'correct_snow': 10353, 'nims_wet': 42290, 'correct_wet': 33933, 'nims_dry': 38000, 'correct_dry': 30547, 'nims_poor_viz': 4575, 'correct_poor_viz': 3409, 'ok': 100665, 'exp_desc': '', 'exp_details': '', 'tracker': ''}


,totalims,correctims,nims_snow_severe,correct_snow_severe,nims_snow,correct_snow,nims_wet,correct_wet,nims_dry,correct_dry,...,exp_desc,exp_details,tracker,overall,perc_dry,perc_poor_viz,perc_snow,perc_snow_severe,perc_wet,avg_recall
0,107445,84301,7635,6059,14945,10353,42290,33933,38000,30547,...,,,,0.784597,0.803868,0.745137,0.69274,0.793582,0.802388,0.767543
